### JSON 파일 -> 'Section' 필드 -> 벡터라이징

In [10]:
import os
from glob import glob
from typing import List, Tuple, Optional
from pathlib import Path

import uuid
import json

import chromadb
import json
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

from dotenv import load_dotenv
import os


In [12]:
def scan_directory(root_path: str) -> List[str]:
    """
    디렉토리를 스캔하여 .md 파일 경로를 찾습니다.
    
    Args:
        root_path (str): 스캔할 루트 디렉토리 경로
    
    Returns:
        List[str]: 발견된 .md 파일의 절대 경로 리스트
    
    Example:
        >>> scan_directory("./manual/user/firstUser")
        ['/path/to/manual/user/firstUser/login/login.md', 
         '/path/to/manual/user/firstUser/approval/approval.md']
    """
    md_files = []
    
    if not os.path.exists(root_path):
        print(f"⚠️ 경로가 존재하지 않습니다: {root_path}")
        return md_files
    
    for root, dirs, files in os.walk(root_path):
        #print(root,dirs,files)
        
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                md_files.append(os.path.abspath(file_path))
    
    return sorted(md_files)

### ✅ 0. 기존 생성된 컬렉션 삭제(데이터 클린징)

In [13]:
client = chromadb.PersistentClient(path="./chroma_db")

collection = client.get_or_create_collection(
    name="manual_user_collection"
)
# 컬렉션 전체 삭제
client.delete_collection("manual_user_collection")

### ✅ 1. 데이터 적재

In [14]:
root_path = './manual/user'
md_files = scan_directory(root_path)
print(f"발견된 파일 수: {len(md_files)}")

for i, filepath in enumerate(md_files, 1):
    print(f"[{i}] {filepath}")


발견된 파일 수: 53
[1] /Users/gu.han/Documents/AI.WORK/RAG_Master/TKS_Q&A_Chatbot/manual/user/accesscontrol/clusterrolebindings/clusterrolebindings.json
[2] /Users/gu.han/Documents/AI.WORK/RAG_Master/TKS_Q&A_Chatbot/manual/user/accesscontrol/clusterroles/clusterroles.json
[3] /Users/gu.han/Documents/AI.WORK/RAG_Master/TKS_Q&A_Chatbot/manual/user/accesscontrol/rolebindings/rolebindings.json
[4] /Users/gu.han/Documents/AI.WORK/RAG_Master/TKS_Q&A_Chatbot/manual/user/accesscontrol/roles/roles.json
[5] /Users/gu.han/Documents/AI.WORK/RAG_Master/TKS_Q&A_Chatbot/manual/user/accesscontrol/serviceaccounts/serviceaccounts.json
[6] /Users/gu.han/Documents/AI.WORK/RAG_Master/TKS_Q&A_Chatbot/manual/user/approval/approval/approval.json
[7] /Users/gu.han/Documents/AI.WORK/RAG_Master/TKS_Q&A_Chatbot/manual/user/bookmark/bookmark/bookmark.json
[8] /Users/gu.han/Documents/AI.WORK/RAG_Master/TKS_Q&A_Chatbot/manual/user/computing/cluster/cluster.json
[9] /Users/gu.han/Documents/AI.WORK/RAG_Master/TKS_Q&A_Chatbo

### 2-1. 데이터 적재 방법(기본 Section 단위)

In [70]:

# 1. 데이터 적재 (Native Chroma)
import chromadb
import json
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

from dotenv import load_dotenv
import os


# 임베딩 함수 정의
embedding_fn = OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),  # 환경변수에서 API 키 읽기
    model_name="text-embedding-3-small"
)

client = chromadb.PersistentClient(path="./chroma_db")

collection = client.get_or_create_collection(
    name="manual_user_collection",
    embedding_function=embedding_fn
)

for i, markdown_filepath in enumerate(md_files, 1):
    with open(markdown_filepath, "r", encoding="utf-8") as f:
        data = json.load(f)

    for i, item in enumerate(data):
        collection.add(
            documents=[item["section"]],
            metadatas=[{
                "manual_type": item["manual_type"],
                "menu_type": item["menu_type"],
                "source_url": item["source_url"],
                "image_urls": json.dumps(item["image_urls"])
            }],
            ids=[f"doc_{uuid.uuid4()}"]
        )

### 2-2. 데이터 적재 방법(Section 병합방법 / 이유: Score가 낮은경우)

OpenAI 임베딩 모델의 성능 sweet spot

text-embedding-3-small / 3-large 모델은 2048 토큰까지 허용

일반적으로 300~800자(한글 기준) = 약 100~300 토큰

👉 이 범위가 의미 밀도와 토큰 길이 간 균형이 좋은 지점

📌 너무 짧으면: 의미 부족 → 유사도 정확도 하락

📌 너무 길면: 압축 손실, 연산 부하, 비효율적 분할

| 기준       | 권장 범위                                    |
| -------- | ---------------------------------------- |
| 영어 기준    | 300–500 tokens (\~1000–2000자)            |
| 한국어 기준   | 500–800자 (문장 기준 4\~8줄)                   |
| 검색 품질 중심 | **500\~800자**가 가장 안정적 (유사도 유지 + 임베딩 안정성) |


In [15]:
import os
import json
import chromadb
import uuid
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# 환경변수 로드
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
assert api_key, "OPENAI_API_KEY 환경변수가 설정되어 있지 않습니다."

# 임베딩 함수 정의
embedding_fn = OpenAIEmbeddingFunction(
    api_key=api_key,
    model_name= "text-embedding-3-small"
)

# Chroma Native 클라이언트 설정
client = chromadb.PersistentClient(path="./chroma_db")

collection = client.get_or_create_collection(
    name="manual_user_collection",
    embedding_function=embedding_fn
)

# ✅ 섹션 병합 함수
def merge_sections(data, chunk_char_limit=800):
    merged = []
    buffer = ""
    image_buffer = []
    
    for item in data:
        section = item.get("section", "").strip()
        if not section:
            continue

        # 누적
        buffer += section + "\n"
        image_buffer += item.get("image_urls", [])

        if len(buffer) >= chunk_char_limit:
            merged.append({
                "section": buffer.strip(),
                "manual_type": item["manual_type"],
                "menu_type": item["menu_type"],
                "source_url": item["source_url"],
                "image_urls": image_buffer.copy()
            })
            buffer = ""
            image_buffer = []

    # 남은 내용 처리
    if buffer.strip():
        merged.append({
            "section": buffer.strip(),
            "manual_type": item["manual_type"],
            "menu_type": item["menu_type"],
            "source_url": item["source_url"],
            "image_urls": image_buffer.copy()
        })

    return merged

########################################################################################
# ✅ 벡터 저장 처리 수행 (데이터 적재)
for markdown_filepath in md_files:
    if not os.path.isfile(markdown_filepath):
        print(f"파일 누락: {markdown_filepath}")
        continue

    with open(markdown_filepath, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError as e:
            print(f"[ERROR] {markdown_filepath} JSON 파싱 실패: {e}")
            continue

    merged_sections = merge_sections(data)
    print(f"{os.path.basename(markdown_filepath)} → {len(merged_sections)} chunks")

    for item in merged_sections:
        collection.add(
            documents=[item["section"]],
            metadatas=[{
                "manual_type": item["manual_type"],
                "menu_type": item["menu_type"],
                "source_url": item["source_url"],
                "image_urls": json.dumps(item.get("image_urls", []))
            }],
            ids=[f"doc_{uuid.uuid4()}"]
        )


clusterrolebindings.json → 2 chunks
clusterroles.json → 2 chunks
rolebindings.json → 2 chunks
roles.json → 2 chunks
serviceaccounts.json → 3 chunks
approval.json → 1 chunks
bookmark.json → 1 chunks
cluster.json → 1 chunks
node.json → 1 chunks
configmaps.json → 2 chunks
hpa.json → 2 chunks
limitranges.json → 2 chunks
priorityclasses.json → 2 chunks
resourcequotas.json → 2 chunks
secrets.json → 2 chunks
dashboard.json → 1 chunks
approval.json → 1 chunks
login.json → 1 chunks
namespaces.json → 2 chunks
project.json → 1 chunks
header.json → 2 chunks
login.json → 1 chunks
namespace.json → 1 chunks
notice.json → 2 chunks
permission.json → 1 chunks
policy.json → 2 chunks
policycondition.json → 2 chunks
project.json → 3 chunks
role.json → 2 chunks
user.json → 2 chunks
namespaces.json → 2 chunks
endpoints.json → 2 chunks
ingressclasses.json → 2 chunks
ingresses.json → 2 chunks
networkpolicies.json → 2 chunks
services.json → 2 chunks
notice.json → 1 chunks
grafana.json → 1 chunks
harbor.json → 1

### ✅ 2-1. 사용자 질문 응답 -> similarity_search_with_score 

In [71]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 동일한 경로로 연동
vectorstore = Chroma(
    collection_name="manual_user_collection",
    persist_directory="./chroma_db",
    embedding_function=embedding
)

query = "신청한 네임스페이스 목록 조회방법"
results = vectorstore.similarity_search_with_score(query, k=3)

for i, (doc, score) in enumerate(results):
    similarity = 1 - score
    print(f"[{i+1}] 유사도 점수 (Similarity): {similarity:.4f}")
    print(f"문서 내용: {doc.page_content[:80]}...")
    print(f"메타데이터: {doc.metadata}")
    print("-" * 40)

[1] 유사도 점수 (Similarity): 0.1280
문서 내용: 신청 조회 ---...
메타데이터: {'image_urls': '[]', 'manual_type': 'firstUser', 'menu_type': 'approval', 'source_url': 'https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/approval'}
----------------------------------------
[2] 유사도 점수 (Similarity): 0.0615
문서 내용: 신청 목록 운영자 유저의 목록 모든 신청 내역이 표시 --- 사용자 유저의 목록 내가 신청한 내역만 표시 신청한 네임스페이스 목록이 표시됩니다....
메타데이터: {'image_urls': '["https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/approvalList.png", "https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/approvalList_ex.png", "https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/list_search_1.png", "https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/list_search_2.png"]', 'manual_type': 'firstUser', 'menu_type': 'approval', 'source_url': 'https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/approval'}
----------------------------------------
[3] 유사도 점수 (Similarity): 0.0103
문서 내용: 3. Namespace 생성 확인 1. 신청관

### ✅ 2-2. 사용자 질문 응답 -> similarity_search(이유: Score 의미 없음, 문맥적 의미에 기반한 데이터 검색만 사용)

In [11]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 동일한 경로로 연동
vectorstore = Chroma(
    collection_name="manual_user_collection",
    persist_directory="./chroma_db",
    embedding_function=embedding
)

query = "신청한 네임스페이스 목록 조회방법"
results = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(results):
    #print(f"문서 내용: {doc.page_content[:80]}...")
    print(f"문서 내용: {doc.page_content}  ")
    print(f"메타데이터: {doc.metadata}")
    print("-" * 40)

문서 내용: 목차1. 신청 조회 - 1.1. 신청 목록 - 1.2. 신청 상세정보 - 1.3. 클러스터 상세정보 2. 신청 결재
신청 조회 ---
신청 목록 운영자 유저의 목록 --- 사용자 유저의 목록 신청한 네임스페이스 목록이 표시됩니다. 진행상태 종류는 아래와 같습니다. 결재 1단계  운영자가 결재하는 단계 결재 2단계  프로젝트 관리자가 결재하는 단계 결재 완료  결재가 완료되어 네임스페이스 생성 반려  반려되어 네임스페이스 신청 거절된 상태 신청내역 목록이라 Namespace 메뉴의 목록과 다를 수 있습니다. 각 필드에 단어가 포함된 목록은 검색됩니다. 아래는 검색 예시 입니다. ---
신청 상세정보 선택한 신청목록의 상세정보를 표시합니다. 신청자, project, cluster, CPU, Memory, Storage 자원 내역 등을 확인할 수 있습니다. 클러스터 상세정보 클릭하면 클러스터의 상세정보를 확인할 수 있습니다. 아래의 권한을 가진 유저는 신청한 네임스페이스 결재할 수 있습니다. 운영자는 결재 1단계에서 승인반려할 수 있습니다. 프로젝트 관리자는 결재 2단계에서 승인반려할 수 있습니다.  
메타데이터: {'image_urls': '["https://doc.tg-cloud.co.kr/manual/console/manual/approval/img/approvalList.png", "https://doc.tg-cloud.co.kr/manual/console/manual/approval/img/approvalList_ex.png", "https://doc.tg-cloud.co.kr/manual/console/manual/approval/img/list_search_1.png", "https://doc.tg-cloud.co.kr/manual/console/manual/approval/img/list_search_2.png", "https://doc.tg-cloud.co.kr/manual/console/manual/approval/img/detail.

### ✅ 3. 조건 필터링 쿼리 (🔍 Native + where)

In [40]:
# 이미지가 존재하는 문서만 조회
# where 조건으로 image_urls 필드가 빈 배열이 아닌 문서 필터링
# include로 documents와 metadatas 데이터만 가져옴
filtered = collection.get(
    where={"menu_type": {"$in": ["namespaces", "approval", "users", "projects"]}},  # menu_type 별로 데이터 조회
    include=["documents", "metadatas"]  # 필요한 필드만 조회
)

# 필터링된 문서와 메타데이터를 순회하며 출력
for doc, meta in zip(filtered["documents"], filtered["metadatas"]):
    print(f"문서 내용 (앞 100자): {doc[:100]}")  # 문서 내용 일부 출력
    print(f"메타데이터: {meta}")  # 메타데이터 전체 출력
    print("=" * 40)  # 구분선 출력


문서 내용 (앞 100자): 신청 조회 ---
메타데이터: {'image_urls': '[]', 'manual_type': 'firstUser', 'menu_type': 'approval', 'source_url': 'https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/approval'}
문서 내용 (앞 100자): 신청 목록 운영자 유저의 목록 모든 신청 내역이 표시 --- 사용자 유저의 목록 내가 신청한 내역만 표시 신청한 네임스페이스 목록이 표시됩니다. 진행상태 종류는 아래와 같습니다. 
메타데이터: {'image_urls': '["https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/approvalList.png", "https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/approvalList_ex.png", "https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/list_search_1.png", "https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/list_search_2.png"]', 'manual_type': 'firstUser', 'menu_type': 'approval', 'source_url': 'https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/approval'}
문서 내용 (앞 100자): 신청 상세정보 선택한 신청목록의 상세정보를 표시합니다. 신청자, project, cluster, CPU, Memory, Storage 자원 내역 등을 확인할 수 있습니다. 클러스터
메타데이터: {'image_urls': '["https://doc.tg-cloud.co.kr/manual/co